# 재구매 고객의 2차 구매 상품

## 라이브러리

In [ ]:
import pandas as pd
from collections import defaultdict
import seaborn as sns
import matplotlib.pyplot as plt
import koreanize_matplotlib # 주피터 노트북 시각화 한글 깨짐 방지
import networkx as nx

## 데이터 로드

In [ ]:
path = "경로" # 파일 경로
df_original = pd.read_excel(path) # 데이터 로드
df = df_original.copy() # raw 유지를 위한 복사
df

## 분석

In [ ]:
# 날짜순으로 정렬
df = df.sort_values(by=['id', 'date'])

# 첫 구매와 다음 구매를 짝짓기
pairs = defaultdict(int)

for customer_id, group in df.groupby('id'):
    products = group['date'].tolist()
    for i in range(len(products) - 1):
        pairs[(products[i], products[i + 1])] += 1

# 결과를 데이터프레임으로 변환
result = pd.DataFrame(list(pairs.items()), columns=['pair', 'count'])
result[['first_purchase', 'next_purchase']] = pd.DataFrame(result['pair'].tolist(), index=result.index)
result.drop(columns=['pair'], inplace=True)

In [ ]:
# 피벗 테이블 생성
pivot_table = result.pivot(index='first_purchase', columns='next_purchase', values='count').fillna(0)

In [ ]:
# 결과 액셀로 내보내기
pivot_table.to_excel("result.xlsx")

## 시각화

In [ ]:
# 데이터 준비
edges = result[result['count'] > 0]  # 횟수가 0이 아닌 항목만 (연결이 되는 항목만)

# 네트워크 그래프 생성
G = nx.from_pandas_edgelist(top_edges, source='first_purchase', target='next_purchase', edge_attr='count')

# 시각화
plt.figure(figsize=(12, 10))
pos = nx.spring_layout(G, k=1)  # k가 클수록 노드 간 간격이 넓어짐

# 노드 그리기
nx.draw_networkx_nodes(G, pos, node_size=500, node_color='skyblue')

# 엣지(선) 그리기 (빈도수에 따라 선 굵기 조절)
weights = [G[u][v]['count'] for u, v in G.edges()]
nx.draw_networkx_edges(G, pos, width=[w * 0.5 for w in weights], edge_color='gray', alpha=0.6)

# 라벨
nx.draw_networkx_labels(G, pos, font_family='Malgun Gothic', font_size=10)

plt.title("상품 간 구매 이동 네트워크")
plt.axis('off')
plt.show()